In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o')
small_llm = ChatOpenAI(model = 'gpt-4o-mini')

In [3]:
from langchain_core.tools import tool

@tool
def add(a:int, b:int)-> int:
    """숫자 a와 b를 더합니다.

    Args:
        a (int): 숫자1
        b (int): 숫자2

    Returns:
        int: 숫자 a와 b를 더한 값
    """
    return a+b

@tool
def multiply(a:int, b:int)-> int:
    """숫자 a와 b를 곱합니다.

    Args:
        a (int): 숫자1
        b (int): 숫자2

    Returns:
        int: 숫자 a와 b를 곱한 값
    """
    return a*b

In [4]:
# 그냥 함수를 호출할 떄는 error 발생 -> tool 이기 떄문에 invoke 매서드 활용
add.invoke({'a':4,'b':8})

12

In [5]:
llm_with_tools = small_llm.bind_tools([add, multiply])

In [6]:
query= '345 곱하고 5993는?'
small_llm.invoke(query)

AIMessage(content='345 곱하기 5993은 2,064,285입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 16, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-505586bd-26ad-4b20-a60d-f06defe16137-0', usage_metadata={'input_tokens': 16, 'output_tokens': 16, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
results = llm_with_tools.invoke(query)
results

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tS0gtfc5c0bcCWbrJeUsdh8u', 'function': {'arguments': '{"a":345,"b":5993}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 154, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9a58120c-605a-40df-95d4-c7243b105b88-0', tool_calls=[{'name': 'multiply', 'args': {'a': 345, 'b': 5993}, 'id': 'call_tS0gtfc5c0bcCWbrJeUsdh8u', 'type': 'tool_call'}], usage_metadata={'input_tokens': 154, 'output_tokens': 18, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
results.tool_calls

[{'name': 'multiply',
  'args': {'a': 345, 'b': 5993},
  'id': 'call_tS0gtfc5c0bcCWbrJeUsdh8u',
  'type': 'tool_call'}]

In [9]:
from typing import Sequence
from langchain_core.messages import AnyMessage, HumanMessage

human_message = HumanMessage(query)
message_list : Sequence[AnyMessage] = [human_message]


In [10]:
ai_message = llm_with_tools.invoke(message_list)

In [11]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_D54ET0yKgMCi83DXizEClUll', 'function': {'arguments': '{"a":345,"b":5993}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 154, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-06756cf9-b5a1-4f8d-a2c9-2135bceba775-0', tool_calls=[{'name': 'multiply', 'args': {'a': 345, 'b': 5993}, 'id': 'call_D54ET0yKgMCi83DXizEClUll', 'type': 'tool_call'}], usage_metadata={'input_tokens': 154, 'output_tokens': 18, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 345, 'b': 5993},
  'id': 'call_D54ET0yKgMCi83DXizEClUll',
  'type': 'tool_call'}]

In [13]:
message_list.append(ai_message)

In [15]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

In [16]:
message_list.append(tool_message)

In [17]:
llm_with_tools.invoke(message_list)

AIMessage(content='345와 5993를 곱한 값은 2,067,585입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 182, 'total_tokens': 202, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2c278bdc-15c0-49ba-a80d-6fe3c29e3c6d-0', usage_metadata={'input_tokens': 182, 'output_tokens': 20, 'total_tokens': 202, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})